In [1]:
# !pip install timm

In [2]:
# !pip install torch==1.10.0+cu111 -f https://download.pytorch.org/whl/cu111/torch_stable.html


In [3]:
# !pip install -U torch torchaudio --no-cache-dir

In [4]:
# ----------------------------
# Prepare training data from Metadata file
# ----------------------------
import pandas as pd
from pathlib import Path

download_path = Path.cwd()/"train_audio"

# Read metadata file
metadata_file = "train_metadata.csv"
df = pd.read_csv(metadata_file)
df.head()

# Construct file path by concatenating fold and file name
# df['relative_path'] = df['primary_label'].astype(str)
df['relative_path'] = + df['filename'].astype(str)


# Take relevant columns
df = df[['relative_path', 'primary_label']]
df.head()


,relative_path,primary_label
0,afrsil1/XC125458.ogg,afrsil1
1,afrsil1/XC175522.ogg,afrsil1
2,afrsil1/XC177993.ogg,afrsil1
3,afrsil1/XC205893.ogg,afrsil1
4,afrsil1/XC207431.ogg,afrsil1


In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
le = preprocessing.LabelEncoder()


df['primary_label'] = le.fit_transform(df['primary_label'])


In [6]:
df

,relative_path,primary_label
0,afrsil1/XC125458.ogg,0
1,afrsil1/XC175522.ogg,0
2,afrsil1/XC177993.ogg,0
3,afrsil1/XC205893.ogg,0
4,afrsil1/XC207431.ogg,0
...,...,...
14847,zebdov/XC629769.ogg,151
14848,zebdov/XC642415.ogg,151
14849,zebdov/XC665873.ogg,151
14850,zebdov/XC666194.ogg,151


In [10]:
!pip install -U torch torchaudio --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 164.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 82.8 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 170.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 227.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 78.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 57.6 MB/s eta 0:00:0000:0100:01


In [11]:
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio

class AudioUtil():
  # ----------------------------
  # Load an audio file. Return the signal as a tensor and the sample rate
  # ----------------------------
    def open(audio_file):
        sig, sr = torchaudio.load(audio_file)
        return (sig, sr)

    def rechannel(aud, new_channel):
        sig, sr = aud

        if (sig.shape[0] == new_channel):
          # Nothing to do
          return aud

        if (new_channel == 1):
          # Convert from stereo to mono by selecting only the first channel
          resig = sig[:1, :]
        else:
          # Convert from mono to stereo by duplicating the first channel
          resig = torch.cat([sig, sig])

        return ((resig, sr))
    def resample(aud, newsr):
        sig, sr = aud

        if (sr == newsr):
          # Nothing to do
          return aud

        num_channels = sig.shape[0]
        # Resample first channel
        resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
        if (num_channels > 1):
          # Resample the second channel and merge both channels
          retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
          resig = torch.cat([resig, retwo])

        return ((resig, newsr))

    def pad_trunc(aud, max_ms):
        sig, sr = aud
        num_rows, sig_len = sig.shape
        max_len = sr//1000 * max_ms

        if (sig_len > max_len):
          # Truncate the signal to the given length
          sig = sig[:,:max_len]

        elif (sig_len < max_len):
          # Length of padding to add at the beginning and end of the signal
          pad_begin_len = random.randint(0, max_len - sig_len)
          pad_end_len = max_len - sig_len - pad_begin_len

          # Pad with 0s
          pad_begin = torch.zeros((num_rows, pad_begin_len))
          pad_end = torch.zeros((num_rows, pad_end_len))

          sig = torch.cat((pad_begin, sig, pad_end), 1)

        return (sig, sr)


    def time_shift(aud, shift_limit):
        sig,sr = aud
        _, sig_len = sig.shape
        shift_amt = int(random.random() * shift_limit * sig_len)
        return (sig.roll(shift_amt), sr)

    def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
        sig,sr = aud
        top_db = 80

        # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
        spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)

        # Convert to decibels
        spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
        return (spec)

    def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
        _, n_mels, n_steps = spec.shape
        mask_value = spec.mean()
        aug_spec = spec

        freq_mask_param = max_mask_pct * n_mels
        for _ in range(n_freq_masks):
          aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

        time_mask_param = max_mask_pct * n_steps
        for _ in range(n_time_masks):
          aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

        return aug_spec
    
    def saveSpectogram(specgram: np.ndarray, filename: str, label: np.uint8, ind: int) -> None:
        bird_name = filename.rsplit('/', 2)[0]
        file_id = filename.rsplit('/', 2)[1].rsplit('.', 2)[0]
        file_name = IMAGES_DIR + 'class_' + str(labels[i]) + '/' + bird_name + '_' + file_id + '_' + str((ind + 1) * 5) + '.png'
        specgram = specgram + 80 # -80 dB -> Min
        specgram = specgram.astype(np.uint8) # 0 - 255 the pixel value
        plt.axis('off')
        plt.imsave(file_name, specgram)

In [12]:
from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio

# ----------------------------
# Sound Dataset
# ----------------------------
class SoundDS(Dataset):
  def __init__(self, df, data_path):
    self.df = df
    self.data_path = str(data_path)
    self.duration = 4000
    self.sr = 44100
    self.channel = 2
    
    self.shift_pct = 0.4
            
  # ----------------------------
  # Number of items in dataset
  # ----------------------------
  def __len__(self):
    return len(self.df)    
    
  # ----------------------------
  # Get i'th item in dataset
  # ----------------------------
  def __getitem__(self, idx):
    # Absolute file path of the audio file - concatenate the audio directory with
    # the relative path
    audio_file = self.data_path + self.df.loc[idx, 'relative_path']
    # Get the Class ID
    class_id = self.df.loc[idx, 'primary_label']

    aud = AudioUtil.open(audio_file)
    # Some sounds have a higher sample rate, or fewer channels compared to the
    # majority. So make all sounds have the same number of channels and same 
    # sample rate. Unless the sample rate is the same, the pad_trunc will still
    # result in arrays of different lengths, even though the sound duration is
    # the same.
    reaud = AudioUtil.resample(aud, self.sr)
    rechan = AudioUtil.rechannel(reaud, self.channel)
    dur_aud = AudioUtil.pad_trunc(rechan, self.duration)
    shift_aud = AudioUtil.time_shift(dur_aud, self.shift_pct)
    sgram = AudioUtil.spectro_gram(shift_aud, n_mels=64, n_fft=1024, hop_len=None)
    aug_sgram = AudioUtil.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)
    spec=AudioUtil.saveSpectogram(aug_sgram,,self.data_path,class_id,

    return aug_sgram, class_id

In [13]:
from torch.utils.data import random_split
data_path='train_audio/'
myds = SoundDS(df, data_path)

# Random split of 80:20 between training and validation
num_items = len(myds)
num_train = round(num_items * 0.8)
num_val = num_items - num_train


In [16]:
import torch.nn.functional as F
from torch.nn import init
import torch.nn as nn
import timm
# ----------------------------
# Audio Classification Model
# ----------------------------
class AudioClassifier (nn.Module):    # ----------------------------
    # Build the model architecture
    # ----------------------------
    def __init__(self, num_classes):
    #define necessary layers
        super().__init__()
        self.num_classes = num_classes      
        self.model = timm.create_model(model_name = "resnet34", pretrained = False,in_chans=2)
        self.model.fc = nn.Linear(self.model.fc.in_features, out_features = num_classes)
        
    def forward(self,X):
        #define forward pass here
        return F.softmax(self.model(X), dim=-1)
   
myModel = AudioClassifier (152).to(DEVICE)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel = myModel.to(device)
# Check that it is on Cuda
next(myModel.parameters()).device

device(type='cpu')

In [17]:
# ----------------------------
# Training Loop
# ----------------------------
def training(model, train_dl, num_epochs):
  # Loss Function, Optimizer and Scheduler
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                                                steps_per_epoch=int(len(train_dl)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')

  # Repeat for each epoch
  for epoch in range(num_epochs):
    running_loss = 0.0
    correct_prediction = 0
    total_prediction = 0

    # Repeat for each batch in the training set
    for i, data in enumerate(train_dl):
        # Get the input features and target labels, and put them on the GPU
        inputs, labels = data[0].to(device), data[1].to(device)

        # Normalize the inputs
        inputs_m, inputs_s = inputs.mean(), inputs.std()
        inputs = (inputs - inputs_m) / inputs_s

        # Zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Keep stats for Loss and Accuracy
        running_loss += loss.item()

        # Get the predicted class with the highest score
        _, prediction = torch.max(outputs,1)
        # Count of predictions that matched the target label
        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]

        #if i % 10 == 0:    # print every 10 mini-batches
        #    print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))

    # Print stats at the end of the epoch
    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches
    acc = correct_prediction/total_prediction
    print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {acc:.2f}')

  print('Finished Training')
  
num_epochs=20   # Just for demo, adjust this higher.
training(myModel, train_dl, num_epochs)

RuntimeError: Failed to open the input "train_audio/comsan/XC613127.ogg" (No such file or directory).

In [ ]:
# ----------------------------
# Inference
# ----------------------------
def inference (model, val_dl):
  correct_prediction = 0
  total_prediction = 0

  # Disable gradient updates
  with torch.no_grad():
    for data in val_dl:
      # Get the input features and target labels, and put them on the GPU
      inputs, labels = (data[0].to(device), data[1].to(device))

      # Normalize the inputs
      inputs_m, inputs_s = inputs.mean(), inputs.std()
      inputs = (inputs - inputs_m) / inputs_s

      # Get predictions
      outputs = model(inputs)

      # Get the predicted class with the highest score
      _, prediction = torch.max(outputs,1)
      # Count of predictions that matched the target label
      correct_prediction += (prediction == labels).sum().item()
      total_prediction += prediction.shape[0]
    
  acc = correct_prediction/total_prediction
  print(f'Accuracy: {acc:.2f}, Total items: {total_prediction}')

# Run inference on trained model with the validation set
inference(myModel, val_dl)

In [ ]:
optimizer = Adam(lr = 0.001, params = model.parameters())

def loss_fn(y_pred, y_true):
    #define your loss function here
    y_pred = torch.clip(y_pred, 1e-8, 1-1e-8)
    l = y_true*torch.log(y_pred)
    # l = l.sum(dim = -1)
    l = l.mean()
    return -l

In [ ]:
def train_on_one_epoch(dataloader, optimizer, loss_fn, len_dataloader):
    #training for one epoch
    
    dataloader = tqdm(dataloader)
    L = 0
    acc = 0
    for i,(x, y) in enumerate(dataloader):
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        y_pred = model(x)
        l = loss_fn(y_pred, y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        L+=l.item()
        acc+=np.sum(y_pred.cpu().detach().numpy().argmax(-1) == y.cpu().detach().numpy().argmax(-1))
    return L/len_dataloader, acc/len_dataloader
        
        

def valid_on_one_epoch(dataloader, loss_fn, len_dataloader):
    #validation for one epoch
    dataloader = tqdm(dataloader)
    L = 0
    acc = 0
    for i,(x, y) in enumerate(dataloader):
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        y_pred = model(x)
        l = loss_fn(y_pred, y)
        L+=l.item()
        acc+=np.sum(y_pred.cpu().detach().numpy().argmax(-1) == y.cpu().detach().numpy().argmax(-1))
        
    return L/len_dataloader, acc/len_dataloader

prev_valid_acc = 0
for epoch in range(30):
    train_loss, train_acc = train_on_one_epoch(train_dl, optimizer, loss_fn, len(train_dataset))
    valid_loss, valid_acc = valid_on_one_epoch(valid_dl, loss_fn, len(valid_dataset))
    print(f"epoch : {epoch} | train loss : {train_loss} | valid_loss : {valid_loss} | train_acc : {train_acc} | valid_acc : {valid_acc}")
    if prev_valid_acc<valid_acc:
        print("model saved..!!")
        torch.save(model.state_dict(), "best_tf_efficientnet_b0.pt")
        prev_valid_acc = valid_acc